In [ ]:
using Random
using Plots
using Distributions
using Statistics
using StatsPlots
using StatsBase
using WAV
using Sound

### Parâmetros gerais

In [ ]:
N = 1000       # Número de amostras
k = 10          # Número de realizações

## Ruído Branco

In [ ]:
Variancia = 3
X_ruido_branco = zeros(k,N)
#X_ruido_branco = sqrt(Variancia)*randn(k,N)
for i in 1 : k
    X_ruido_branco[i,:] .= sqrt(Variancia)*randn(N)
end
    
size(X_ruido_branco)
    


In [ ]:
p1 = plot(X_ruido_branco[1,:], label="Ruido_branco", legend=false, size = (1150, 500))

In [ ]:
# Tocar aúdio wavplay(y, fs)
wavplay(X_ruido_branco[1,:],8000)

In [ ]:
#R = (vários vetores..)
Lag_max = 100
Lags = 0:Lag_max-1
#J = length(Lags)
R = zeros(length(Lags),k)

for i in 1:k
    R[:,i] .= autocor(X_ruido_branco[i,:], Lags; demean=true) #Autocorrelação do vetor X
end

R_medio = mean(R,dims=2)


In [ ]:
pr = plot(R_medio, line = :stem, marker = :circle, markersize = 5, linewidth = 3, size = (1100,500), label = "ACF", legend = false)  

## Sequência binária aleatória

In [ ]:
prob = 1/3 # Probabilidade de geração de um '1'

X_binaria = rand(Bernoulli(prob), k, N)


In [ ]:
p3 = plot(X_binaria[1,:], line = :stem, marker = :circle, markersize = 5, linewidth = 3, size = (1100,500), label = "Bernoulli", legend = false) 

In [ ]:
#R = (vários vetores..)
Lag_max = 100
Lags = 0:Lag_max-1
#J = length(Lags)
R_binaria = zeros(length(Lags),k)

for i in 1:k
    R_binaria[:,i] .= autocor(X_binaria[i,:], Lags; demean=true) #Autocorrelação do vetor X
end

R_medio_binaria = mean(R_binaria,dims=2)


In [ ]:
pb = plot(R_medio_binaria, line = :stem, marker = :circle, markersize = 5, linewidth = 3, size = (1100,500), label = "ACF", legend = false) 

### Processo de Poisson

In [ ]:
# Função para gerar tempos de chegada de um processo de Poisson
function poisson_process(lambda::Float64, T::Float64)
    arrival_times = Float64[]
    t = 0.0
    
    while t < T
        t += rand(Exponential(1/lambda))
        if t < T
            push!(arrival_times, t)
        end
    end
    
    return arrival_times
end

# Parâmetros
lambda = 1.0 # taxa do processo de Poisson (eventos por unidade de tempo)
T = 150.0     # tempo total de simulação

# Simulação do processo de Poisson
arrival_times = poisson_process(lambda, T)

In [ ]:
# Plotando o processo de Poisson
plot([0; arrival_times], 0:length(arrival_times), label="Processo de Poisson", marker=:o)
xlabel!("Tempo")
ylabel!("Número de eventos")
title!("Simulação de um Processo de Poisson")

In [ ]:
#R = (vários vetores..)
Lag_max = 100
Lags = 0:Lag_max-1
#J = length(Lags)
R_Poisson = zeros(length(Lags),k)

for i in 1:k
    R_Poisson[:,i] .= autocor(arrival_times, Lags; demean=true) #Autocorrelação do vetor X
end

R_medio_Poisson = mean(R_Poisson,dims=2)

In [ ]:
ppoisson = plot(R_medio_Poisson, line = :stem, marker = :circle, markersize = 5, linewidth = 3, size = (1100,500), label = "ACF", legend = false) 

### Scatter plot (Processo de Poisson)

In [ ]:
Tau = 100
J = length(arrival_times)
#J = length(X_ruido_branco[1,:])


Dados_1 = zeros(J-Tau,1)
Dados_2 = zeros(J-Tau,1)


Dados_1 .= arrival_times[1:J-Tau,]
Dados_2 .= arrival_times[1+Tau:end,1]

In [ ]:
p5 = scatter(Dados_2, Dados_1, label="Dados", size=(600,600), legend=false)
xlabel!("x(n-τ)")
ylabel!("x(n)")
display(p5)

### Scatter plot - Processo de AR(1) 

In [ ]:
J = N
Coef = 0.7
Tau = 5
Dados = randn(J,1)

for j in 2:J
    Dados[j,1] = Coef * Dados[j-1,1] + randn()
end

Dados_1 = zeros(J-Tau,1)
Dados_2 = zeros(J-Tau,1)

Dados_1 .= Dados[1:J-Tau]
Dados_2 .= Dados[1+Tau:end]

In [ ]:
p5 = scatter(Dados_2, Dados_1, label="Dados", size=(600,600), legend = false)
xlabel!("x(n-τ)")
ylabel!("x(n)")
display(p5)

#### Funções de áudio (similar ao Matlab)

In [ ]:
S = 8192 # sampling rate in Hz
x = 0.6 * cos.(2π*(1:S÷2)*440/S)
y = 0.7 * sin.(2π*(1:S÷2)*660/S)
sound(x, S) # monophonic
sound([x y], S) # stereo
